In [ ]:
%cd E:\


In [ ]:
%cd E:\Medical-Chatbot-Gen-AI


In [ ]:
%pwd

In [ ]:
from os import chdir
chdir("E:\\Medical-Chatbot-Gen-AI")

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  


In [ ]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()   
    return documents

In [ ]:
extracted_doc = load_pdf_file("Data/")

In [ ]:
len(extracted_doc)

In [ ]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(Document(page_content=doc.page_content, metadata={"source": src}))  
    return minimal_docs

In [ ]:
minimal_docs = filter_to_minimal_docs(extracted_doc)

In [ ]:
minimal_docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks


In [ ]:
text_chunks = text_split(minimal_docs)
print("Length of text chunks:", len(text_chunks))

In [ ]:
text_chunks 

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [ ]:
embedding = download_embeddings()

In [ ]:
embedding

In [ ]:
print(len(embedding.embed_query("hello")))

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
from pinecone import Pinecone
pincecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pincecone_api_key)

In [ ]:
pc

In [ ]:
# from pinecone import ServerlessSpec

index_name = "medical-chatbot"

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=384,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )

index = pc.Index(index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

# vector_store = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     embedding=embedding,
#     index_name=index_name,
# )

In [ ]:
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name=index_name,
)

In [ ]:
# add more data to the existing index
# docsearch.add_documents(documents=[list of new documents])

In [ ]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
retrieved_docs = retriver.invoke("what is fever?")
retrieved_docs

In [ ]:
from groq import Groq
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])


In [ ]:
def retrieve_context(query, top_k=3):
    # Change .encode() to .embed_query()
    vector = embedding.embed_query(query) 
    res = index.query(
        vector=vector,
        top_k=top_k,
        include_metadata=True
    )
    return "\n\n".join(m["metadata"]["text"] for m in res["matches"])

In [ ]:
def ask_groq(question, context):
    # context = context[:6000]
    prompt = f"""
You are a Community Health Assistant for rural areas. Use very simple, supportive language.
Provide clear, concise health facts based on the context below. Follow these rules strictly:

1. LANGUAGE & TONE
The 10-Year-Old Rule: Explain facts as if speaking to a child. Use "belly" for abdomen, "breathing" for respiratory, and "signs" for symptoms. Avoid all Latin or Greek roots (e.g., use "heart," not "cardiac").
Action-Oriented: Use simple verbs like "Drink," "Rest," and "Wash."
No Extra Details: Provide only the core instruction. For example, say "Rest in a room" instead of "Rest in a quiet, dark room."
Strict Neutrality: No emojis. No emotional fillers like "Don't panic," "I am sorry," or "This is worrying."

2. STRUCTURE & GREETING
Direct Start: If th user asks a question, start the facts immediately. Only say "Hello!" or "Hi!" if the user greeted you first.
No Labels: Never use internal labels like "Immediate Focus:" or "Clinical Data Phase:".
Formatting: Use simple bullet points for health facts. Do not use headings if the information is missing.

3. CONTENT LIMITS
Scope: Focus ONLY on home care and first aid. Do not include statistics, regional data, or mentions of schools/workplaces.
Missing Information: If the answer is not in the context, say only: "I do not have much information about this problem. Please ask a health worker."

4. MANDATORY FOOTER
Always include a section titled '⚠️ WHEN TO SEE A DOCTOR' listing clear warning signs.
Always include this exact disclaimer: 'This is for information only. Please speak with a healthcare provider for medical diagnosis.'

Context:
{context}

Question:
{question}
"""


    response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct", # Updated Model ID
    messages=[{"role": "user", "content": prompt}],
    temperature=0
    )   

    return response.choices[0].message.content


In [ ]:
def rag_answer(question):
    context = retrieve_context(question)
    return ask_groq(question, context)


In [ ]:
print(rag_answer("What are the symptoms of croup?"))


In [ ]:
print(rag_answer("How to treat and reduce cold? i have cold from 2 days what should i do? "))

In [ ]:
print(rag_answer("What is Cephalosporins?"))

In [ ]:
print(rag_answer("What is Cerebral aneurysm?"))

In [ ]:
print(rag_answer("I am suffering from cold since 3 days, and i want to know the difference betwween cold and sinus infection and give me the symptomes for both conditions." ))

In [ ]:
print(rag_answer("What are the desiseases that are comman during winter season"))

In [ ]:
print(rag_answer("What are the causes of headache? and how to treat it? "))

In [ ]:
print(rag_answer("I have a very bad cough and my chest feels tight. My school says many students are staying home because of the cold weather in the US. What should I do to get better? Thanks!"))

In [ ]:
print(rag_answer("My young child has a very high fever and has a barking cough that sounds like a dog. We usually go to the school football games on Fridays, and everyone in our town is talking about the new clinic opening in the city. What can I do at home for this cough? Thank you so much for your help!"))

TestCase-1: The "US/Neighbor Statistics" Test

In [ ]:
print(rag_answer("My neighbor told me that in the United States, thousands of people go to the doctor for the common cold during the winter. My throat is very sore and I have a dry cough. What can I do at home? Thanks!" ))

Test Case 2: The "Rural Safety/Emergency" Test

In [ ]:
print(rag_answer("I am very worried. My child's breathing is very fast and making a whistling sound. We have a farm and the nearest clinic is far. Is there something I can do or do I need to leave now?"))

Test Case 3: The "Simple Language" Test

In [ ]:
print(rag_answer("Hello, I have a gastrointestinal issue and abdominal pain. Should I take pharmacological medication or just drink water? Thank you."))

Test Case 4: The "Irrelevant Activity" Test

In [ ]:
print(rag_answer("I have a bad headache. My friends are going to the football game at the high school tonight, but I feel dizzy. What is the home care for this? Thanks!"))

Test Case 5: The "No Greeting" Test (Direct Question)

In [ ]:
print(rag_answer("What are the home treatments for a minor skin burn from a cooking fire?"))

In [ ]:
print(rag_answer("Patients presenting with cephalalgia  should maintain strict recumbency in a profoundly quiet and dark room to mitigate sensory overload. It is imperative to monitor for neurological deficits."))